In [ ]:
# ============================================================
# INFERENCIA "AUTO"(regresión / binaria / multiclase)
# Entradas:
#   1) resultados.zip   (contiene modelo.keras + metadata.json)
#   2) New_final.csv    (nuevos datos con las MISMAS features)
# Salida:
#   New_predicciones.csv (agrega columnas según el tipo de problema)
#
# Detección automática:
#   - Regresión:     salida (None, 1) y última activación ~ linear
#   - Binaria:       salida (None, 1) y última activación ~ sigmoid
#   - Multiclase:    salida (None, C) con C>1 y última activación ~ softmax
# ============================================================

import os
import json
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# ----------------------------
# 0) Config
# ----------------------------
ZIP_RESULTADOS = "resultados.zip"
NEW_CSV = "New_final.csv"
OUT_PRED = "New_predicciones.csv"

EXTRACT_DIR = "inferencia_artifacts"
os.makedirs(EXTRACT_DIR, exist_ok=True)

# ----------------------------
# 1) Descomprimir resultados
# ----------------------------
with zipfile.ZipFile(ZIP_RESULTADOS, "r") as z:
    z.extractall(EXTRACT_DIR)

model_path = os.path.join(EXTRACT_DIR, "modelo.keras")
meta_path  = os.path.join(EXTRACT_DIR, "metadata.json")

# ----------------------------
# 2) Cargar modelo + metadata
# ----------------------------
model = keras.models.load_model(model_path)

with open(meta_path, "r", encoding="utf-8") as f:
    meta = json.load(f)

n0_esperado = int(meta.get("n_features", -1))
# Umbral opcional (si lo guardaste alguna vez). Si no, 0.5.
threshold = float(meta.get("threshold", 0.5))

# ----------------------------
# 3) Cargar nuevos datos
# ----------------------------
new = pd.read_csv(NEW_CSV)

# Si viene target, se quita para inferencia (aunque dijiste que no hay conflicto)
if "target" in new.columns:
    new_X = new.drop(columns=["target"])
else:
    new_X = new.copy()

# ----------------------------
# 4) Validación mínima de columnas
# ----------------------------
if n0_esperado != -1 and new_X.shape[1] != n0_esperado:
    raise ValueError(
        f"New_final.csv tiene {new_X.shape[1]} features, pero el modelo espera {n0_esperado}. "
        "Asegúrate de aplicar el mismo preprocesamiento y tener las mismas columnas."
    )

non_numeric = [c for c in new_X.columns if not pd.api.types.is_numeric_dtype(new_X[c])]
if non_numeric:
    raise ValueError(
        f"Estas columnas no son numéricas: {non_numeric}. "
        "New_final.csv debe venir ya preprocesado (solo numérico)."
    )

X = new_X.astype(np.float32).values

# ----------------------------
# 5) Predicción cruda
# ----------------------------
y_raw = model.predict(X, verbose=0)

# ----------------------------
# 6) Detección automática del tipo de problema
# ----------------------------
# Dimensión de salida (None, out_dim)
out_shape = model.output_shape
out_dim = int(out_shape[-1]) if isinstance(out_shape, tuple) else int(out_shape[0][-1])

# Activación de la última capa (si existe)
last_layer = model.layers[-1]
act_name = None
try:
    act = getattr(last_layer, "activation", None)
    act_name = act.__name__ if act is not None else None
except Exception:
    act_name = None

def detect_task(out_dim: int, act_name: str | None) -> str:
    act_name = (act_name or "").lower()
    if out_dim == 1:
        if "sigmoid" in act_name:
            return "binary_classification"
        # si no es sigmoid, lo tratamos como regresión (lineal u otra)
        return "regression"
    else:
        # C>1
        if "softmax" in act_name:
            return "multiclass_classification"
        # fallback: si sale vector, normalmente se usa como multiclase (aunque no tenga softmax)
        return "multiclass_classification"

task = detect_task(out_dim, act_name)

print("=== Detección automática ===")
print("output_shape:", model.output_shape)
print("last_activation:", act_name)
print("task_detected:", task)

# ----------------------------
# 7) Post-procesamiento según tarea + guardar
# ----------------------------
out = new.copy()

if task == "regression":
    # y_raw: (n, 1) o (n,)
    y_pred = np.asarray(y_raw).reshape(-1)
    out["y_pred"] = y_pred

elif task == "binary_classification":
    # y_raw: (n, 1) probabilidades
    y_prob = np.asarray(y_raw).reshape(-1)
    y_class = (y_prob >= threshold).astype(int)
    out["y_prob"] = y_prob
    out["y_class"] = y_class

elif task == "multiclass_classification":
    # y_raw: (n, C) probabilidades o scores
    y_prob = np.asarray(y_raw)
    if y_prob.ndim != 2 or y_prob.shape[1] != out_dim:
        raise ValueError(f"Salida inesperada para multiclase. shape={y_prob.shape}, out_dim={out_dim}")

    y_class = y_prob.argmax(axis=1).astype(int)

    # Guardar probs por clase (columnas prob_c0, prob_c1, ...)
    for j in range(out_dim):
        out[f"prob_c{j}"] = y_prob[:, j]

    out["y_class"] = y_class

else:
    raise ValueError(f"Tarea no soportada: {task}")

out.to_csv(OUT_PRED, index=False)
print(f"\n✔ Predicciones guardadas en: {OUT_PRED}")
print(out.head())
